In [25]:
def get_time(minute_after_midnight):
    # hh:mm
    minute_after_midnight += 480 # 8:00
    hh = minute_after_midnight // 60
    mm = minute_after_midnight % 60
    return str(hh) + ":" + str(mm).zfill(2)


In [26]:
import import_ipynb
from dijkstra import nodes, arc_matrix, get_shortest_path, format_path
from tsp_costruttivo import build_tsp_path

In [27]:
import random
import numpy as np

random.seed(0)

MINUTES_IN_A_DAY = 24*60
MINUTES_PER_WORK_CYCLE = 120
LAB = nodes.index('7')
MASSIMA_CAPACITA_TRASPORTABILE = 6

distribuzione_tamponi_al_minuto = np.array([random.randint(0, 10) for _ in range(MINUTES_IN_A_DAY)])

distribuzione_reparti = np.array([random.random() for _ in range(len(nodes))])
junction_nodes_start_index = nodes.index('X1')
distribuzione_reparti[junction_nodes_start_index:] = 0
# distribuzione_reparti /= distribuzione_reparti.sum()
# print(distribuzione_reparti)

print(distribuzione_tamponi_al_minuto)
print(distribuzione_reparti)

[6 6 0 ... 3 0 5]
[0.5290949  0.94135742 0.68025796 0.630908   0.62781515 0.49698971
 0.73091927 0.24919444 0.89175426 0.27447266 0.94494501 0.92649671
 0.07792452 0.4481797  0.74403628 0.44965407 0.50889902 0.80682394
 0.70499216 0.95800422 0.16448599 0.92355929 0.92798625 0.63474894
 0.94039083 0.25268559 0.88178728 0.77347929 0.609689   0.09062924
 0.03013435 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


In [28]:
## funzioni di aggiornamento dei tamponi
def aggiornamento_tamponi(tamponi_per_reparto, current_minute):
    return aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute)

# ad-hoc
def aggiornamento_tamponi_semplice(tamponi_per_reparto, current_minute):
    
    if(current_minute==1):
        reparto = nodes.index('22')
        tamponi_per_reparto[reparto] += 7
        print("Arrivati 7 tamponi al reparto 22")
    
    if(current_minute==11):
        reparto = nodes.index('12')
        tamponi_per_reparto[reparto] += 12
        print("Arrivati 12 tamponi al reparto 12")
    
    return tamponi_per_reparto

# tramite istanza giornaliera
def aggiornamento_tamponi_istanza(tamponi_per_reparto, current_minute):
    # DA IMPLEMENTARE
    return tamponi_per_reparto

# tramite distribuzione statistica
def aggiornamento_tamponi_statistica(tamponi_per_reparto, current_minute):
    nuovi_tamponi_per_reparto = (distribuzione_tamponi_al_minuto[current_minute] * distribuzione_reparti).astype(int)
    tamponi_per_reparto += nuovi_tamponi_per_reparto

    return tamponi_per_reparto


In [29]:
## funzioni di pianificazione in caso di assenza di tamponi
def pianifica_no_tamponi(tamponi_per_reparto, start, current_minute):
    # return pianifica_goto_lab(tamponi_per_reparto, start, current_minute)
    pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute)

# nessun piano
def pianifica_rimani_fermo(tamponi_per_reparto, start, current_minute):
    return None

# ritorno al laboratorio
def pianifica_goto_lab(tamponi_per_reparto, start, current_minute):
    end = LAB
    
    (path, distance) = get_shortest_path(arc_matrix, start, end)
    
    path = path[1:]
    return path


In [30]:
def get_complete_path(path_grafo_ridotto, current_position):
    path = [current_position]
    for node in path_grafo_ridotto:
        (path_to_node, _) = get_shortest_path(arc_matrix, current_position, node)
        # print("current_position", nodes[current_position])
        # print("destination node", nodes[node])
        # print("path_to_node", format_path(path_to_node))
        path.extend(path_to_node[1:])
        current_position = node
    
    path = path[1:]
    return path

# current_position = nodes.index('X1')
# path = [nodes.index('22'), nodes.index('9')]

# print("Current position:", nodes[current_position])
# print("Path:", format_path(path))

# path_completo = get_complete_path(path, current_position)
# print("Complete path:", format_path(path_completo))

In [31]:
## funzioni di pianificazione (lista di nodi nell'ordine in cui si vogliono visitare)
def pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati, moving_info):
    # return pianifica_gotomax(tamponi_per_reparto, current_position, current_minute)
    
    if(np.sum(tamponi_per_reparto)==0):
        return pianifica_no_tamponi(tamponi_per_reparto, current_position, current_minute)
    
    current_work_cycle_minute = current_minute % MINUTES_PER_WORK_CYCLE
    residual_time = MINUTES_PER_WORK_CYCLE - current_work_cycle_minute
    path_grafo_ridotto = build_tsp_path(arc_matrix, tamponi_per_reparto, residual_time, tamponi_trasportati, current_position, LAB, moving_info=moving_info)
    path = get_complete_path(path_grafo_ridotto, current_position)
    return path

# si va al reparto con più tamponi da raccogliere, il piano è calcolato con Dijkstra
def pianifica_gotomax(tamponi_per_reparto, start, current_minute):
    end = np.argmax(tamponi_per_reparto)
    
    if tamponi_per_reparto[end] == 0: # non ci sono tamponi da raccogliere
        return pianifica_no_tamponi(tamponi_per_reparto, start, current_minute)

    (path, distance) = get_shortest_path(arc_matrix, start, end)

    path = path[1:]
    return path


In [32]:
## funzioni di valutazione dei piani
def get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time):
    return get_score_density(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time)

# score = somma dei tamponi da raccogliere diviso il tempo di percorrenza (tamponi/minuto)
def get_score_density(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time):
    if piano in (None, []):
        return 0

    # calcolo tamponi
    tamponi_nel_path = sum(tamponi_per_reparto[p] for p in piano)
    tamponi_nel_path = min(tamponi_nel_path, MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati)
    
    # calcolo distanza
    distanza = arc_matrix[current_position][piano[0]]
    for i in range(len(piano)-1):
        distanza += arc_matrix[piano[i]][piano[i+1]]
    distanza -= moving_time
    
    # print("tamponi", score, "distanza", distanza)
    score = tamponi_nel_path / distanza
    return score

In [33]:
def reconsider(tamponi_per_reparto, current_position, current_minute, piano, tamponi_trasportati, moving_info):
    (_,_,moving_time) = moving_info
    score = get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano, moving_time)
    
    piano_alternativo = pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati, moving_info=moving_info)
    
    if piano_alternativo in (None, []):
        return False, None
    
    moving_time_alternativo = moving_time
    if(piano not in (None, []) and piano[0] != piano_alternativo[0]):
        moving_time_alternativo = -moving_time
    score_alternativo = get_score(tamponi_per_reparto, current_position, tamponi_trasportati, piano_alternativo, moving_time_alternativo)
    
    # if score_alternativo > score:
    #     print(f"piano originale: {format_path(piano)} score: {score}")
    #     print(f"piano alternativo: {format_path(piano_alternativo)} score: {score_alternativo}")
        
    return score_alternativo > score, piano_alternativo

In [34]:
def move_toward(current_position, next_position, moving_info):
    (_,_,moving_time) = moving_info
    distance = int(get_shortest_path(arc_matrix, current_position, next_position)[1])
    moving_time = min(moving_time+1, distance)
    print(f"Moving from {nodes[current_position]} to {nodes[next_position]} ... {moving_time}/{distance}")
    moving_info = (current_position, next_position, moving_time)
    return distance, moving_info

In [35]:
def get_status(current_minute, current_position, tamponi_trasportati):
    return f"Minuto {current_minute} - {get_time(current_minute)} Current position: {nodes[current_position]} Tamponi trasporati: {tamponi_trasportati}"

In [36]:
tamponi_per_reparto = np.zeros(len(nodes)).astype(int)
current_position = LAB
current_minute = 0
moving_info = (None, None, 0)

tamponi_trasportati = 0
tamponi_in_laboratorio = 0

last_minute = 120

while current_minute < MINUTES_IN_A_DAY:
    print(get_status(current_minute, current_position, tamponi_trasportati))
        
    tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
    # print(tamponi_per_reparto)

    piano = pianifica(tamponi_per_reparto, current_position, current_minute, tamponi_trasportati, moving_info)
    print(f"Piano: {format_path(piano)}")

    while piano not in [[], None]:
        next_position = piano[0]
        distance, moving_info = move_toward(current_position, next_position, moving_info)
        # print(moving_info)
        (_,_,moving_time) = moving_info
        current_minute += 1
        
        if distance == moving_time: # arrivato a destinazione
            print(f"Arrivato al nodo {nodes[next_position]}")
            current_position = next_position
            piano = piano[1:]
            moving_info = (None, None, 0)
            
            # raccolta tamponi
            if tamponi_per_reparto[current_position] > 0 and min(tamponi_per_reparto[current_position], MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati) > 0:
                tamponi_raccolti = min(tamponi_per_reparto[current_position], MASSIMA_CAPACITA_TRASPORTABILE - tamponi_trasportati)
                print(f"Tamponi raccolti: {tamponi_raccolti}")
                tamponi_trasportati += tamponi_raccolti
                tamponi_per_reparto[current_position] -= tamponi_raccolti
            
            # arrivo al laboratorio
            if current_position == LAB:
                tamponi_in_laboratorio += tamponi_trasportati
                print(f"Tamponi totali in laboratorio: {tamponi_in_laboratorio}")
                tamponi_trasportati = 0
        
        print()
        print(get_status(current_minute, current_position, tamponi_trasportati))
        
        tamponi_per_reparto = aggiornamento_tamponi(tamponi_per_reparto, current_minute)
        # print(tamponi_per_reparto)

        must_reconsider, piano_alternativo = reconsider(tamponi_per_reparto, current_position, current_minute, piano, tamponi_trasportati, moving_info)
        if must_reconsider:
            print(f"Riconsidero il piano {format_path(piano)} con il piano alternativo {format_path(piano_alternativo)}")
            # considero inversione del percorso su corridoio
            if(piano not in (None, []) and piano[0] != piano_alternativo[0]):
                (moving_from, moving_toward, moving_time) = moving_info
                moving_info = (moving_toward, moving_from, -moving_time)
            piano = piano_alternativo
        
        print(f"Piano: {format_path(piano)}")
        
        ## TESTING
        if current_minute > last_minute:
            break
        ##
    
    ## TESTING
    if current_minute > last_minute:
        break
    ##

    current_minute += 1
    print()

Minuto 0 - 8:00 Current position: 7 Tamponi trasporati: 0
Piano: No path

Minuto 1 - 8:01 Current position: 7 Tamponi trasporati: 0
Arrivati 7 tamponi al reparto 22
Piano: X17 -> X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from 7 to X17 ... 1/4

Minuto 2 - 8:02 Current position: 7 Tamponi trasporati: 0
Piano: X17 -> X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from 7 to X17 ... 2/4

Minuto 3 - 8:03 Current position: 7 Tamponi trasporati: 0
Piano: X17 -> X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from 7 to X17 ... 3/4

Minuto 4 - 8:04 Current position: 7 Tamponi trasporati: 0
Piano: X17 -> X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from 7 to X17 ... 4/4
Arrivato al nodo X17

Minuto 5 - 8:05 Current position: X17 Tamponi trasporati: 0
Piano: X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from X17 to X18 ... 1/3

Minuto 6 - 8:06 Current position: X17 Tamponi trasporati: 0
Piano: X18 -> X19 -> 22 -> X19 -> X18 -> X17 -> 7
Moving from X17 to X18 ... 2/3

Minuto 7